In [52]:
import numpy as np
import pandas as pd
import pickle
import psutil
import random as rn
import tensorflow as tf
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [2]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 
from keras import backend as K

Using TensorFlow backend.


In [3]:
with open('lime_analysis_data/complete_data.pkl', 'rb') as handle:
    complete_data = pickle.load(handle)

In [17]:
with open('lime_analysis_data/results.pkl', 'rb') as handle:
    lime_dict = pickle.load(handle)

In [4]:
lime_dict['intensity_df']

In [5]:
x_train = complete_data['xtrain']
x_test = complete_data['xtest']
x_val = complete_data['xval']
y_train = complete_data['ytrain']
y_test = complete_data['ytest']
y_val = complete_data['yval']
# param_dict = complete_data['params_dict']

In [6]:
def predict(qc):
    global best_model
    qc = best_model.predict(qc)
    return qc.reshape(qc.shape[0])

In [7]:
from keras.models import load_model
best_model = load_model('lime_analysis_data/train_v21.h5')

In [28]:
x_valid = x_val.head(2).copy()

In [9]:
selected_cols = x_train.columns

In [32]:
x_valid.reset_index(drop=True, inplace=True)

In [14]:
import lime
import lime.lime_tabular
#explainer = lime.lime_tabular.LimeTabularExplainer(np.array(x_train), feature_names=selected_cols[:383])
explainer = lime.lime_tabular.LimeTabularExplainer(np.array(x_train), feature_names=selected_cols, class_names=[], verbose=True, mode='regression',random_state=42)
# i = np.random.randint(0, np.array(x_valid).shape[0])
# exp = explainer.explain_instance(x_valid.loc[10,:], predict, num_features=383)
# exp = explainer.explain_instance(x_valid.loc[0].values, model.predict, num_features=100)

In [45]:
def mul(im):
    
#     for im in range(3):
    print('---------------',im,'--*-------------')
    exp = explainer.explain_instance(x_valid.loc[im], predict, num_features=x_valid.shape[0])
    print('sdjdbhhjdsbjsdbjsdbdsjb')
    name_pos = list(x_valid.columns)
    intensity = [0]*len(name_pos)
    try:
        for i in exp.as_list():
            print(im,'ajbhjb')
            if i[0].find(' < ') != -1 and i[0].find(' <= ') != -1:
                intensity[name_pos.index(i[0][i[0].find(' < ')+3:i[0].find(' <= ')])] = i[1]
            elif i[0].find(' <= ') != -1 and i[0].find(' < ') != -1:
                intensity[name_pos.index(i[0][i[0].find(' <= ')+4:i[0].find(' < ')])] = i[1]
            elif i[0].find(' < ') != -1:
                intensity[name_pos.index(i[0][0:i[0].find(' < ')])] = i[1]
            elif i[0].find(' <= ') != -1:
                intensity[name_pos.index(i[0][0:i[0].find(' <= ')])] = i[1]
            elif i[0].find(' > ') != -1:
                intensity[name_pos.index(i[0][0:i[0].find(' > ')])] = i[1]
            elif i[0].find(' >= ') != -1:
                intensity[name_pos.index(i[0][0:i[0].find(' >= ')])] = i[1]
    except:
        print('except')
        pass
    return {''.format(im):intensity}

In [48]:
import threading

In [24]:
import time

In [41]:
def featur_im(explainer,x_valid):
    input_list = list(range(x_valid.shape[0]))
    start = time.time()
    p = Pool(processes = 6)
    data1 = p.map(mul,input_list)
#     p.join()
    p.close()
    print('Took: {} seconds'.format(time.time()-start))
#     if im == 0:
#         intensity_dic = {'feature_name':name_pos, 'intensity0':intansity}
#         df_int = pd.DataFrame(intensity_dic)
#     else:
#         df_int['intensity'+str(im)] = intansity
    return data1

In [49]:
def featur_im1(explainer,x_valid):
    input_list = list(range(x_valid.shape[0]))
    start = time.time()
    data1 = threading.Thread(target=mul,args=(input_list))
#     p.join()
#     p.close()
    print('Took: {} seconds'.format(time.time()-start))
#     if im == 0:
#         intensity_dic = {'feature_name':name_pos, 'intensity0':intansity}
#         df_int = pd.DataFrame(intensity_dic)
#     else:
#         df_int['intensity'+str(im)] = intansity
    return data1

In [51]:
a  = featur_im(explainer,x_valid)

--------------- 0 --*-------------
--------------- 1 --*-------------


Process ForkPoolWorker-122:
Process ForkPoolWorker-121:
Process ForkPoolWorker-120:
Process ForkPoolWorker-123:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiproc

In [61]:
a

,0,1,2,3,4,5,6,7,8,9,...,386,387,388,389,390,391,392,393,394,395
feature_name,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190
intensity0,0,0,0,-1050.02,1593.6,0,0,0,-9750.5,-10218.1,...,0,1665.84,-1195.67,-1123.61,0,123.669,0,2842.9,-7868.63,0
intensity1,0,-2877.11,0,0,-1284.06,0,0,-31531.5,-6510.51,987.884,...,0,0,0,3648.35,1275.07,-54.0238,-1058.93,2921.55,-7718.21,0
intensity2,0,4162.12,0,637.31,0,0,-3665.32,-14283.4,-7156.27,0,...,0,-3525.86,5481.31,0,-4050.15,2683.65,-1517.19,709.982,0,0


In [62]:
lime_dict['intensity_df']

,0,1,2,3,4,5,6,7,8,9,...,386,387,388,389,390,391,392,393,394,395
feature_name,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190
intensity0,-847.216,2107.86,4585.79,-577.805,1077.1,-1097.03,-1176.54,3935.76,-6079.17,-9331.25,...,2430.49,911.9,541.985,3098.9,2613.4,-2190.1,1727.73,-908.552,-5901.92,5081.44
intensity1,516.196,-2420.48,1917.38,1541.52,129.452,-707.282,3804.4,-32065.7,-7176.48,1533.95,...,-863.125,-1344.11,-237.036,-2524.54,-8235.41,6408.19,82.961,271.691,-9445.07,611.325
intensity2,-269.659,1020.19,-674.818,-108.316,-3087.72,-568.525,-4471.39,-16375.2,-8697.19,1674.97,...,-668.692,491.943,4030.31,-6917.3,8063.27,7150.93,-3789.86,-519.065,3499.75,1076.9
intensity3,1201.92,-1319.68,731.332,653.774,-228.053,-1461.8,5933.27,2977.94,-7836.87,1837.28,...,-1309.65,-1108.64,2737.38,1862.82,-8256.8,8380.8,1697.55,-555.872,831.474,3599.44
intensity4,-221.768,-209.689,2030.12,2027.34,1751.28,1284.74,-5101.71,4001.41,-7480.61,-16698.9,...,-212.559,-4009.1,-9245.04,-1503.56,-1502.37,-528.616,-577.006,-3893.6,9600.44,3821.55
intensity5,968.653,-1639.03,1089.52,-1526.99,-1806.29,-30.4648,21.7861,44895.5,-5111.71,-19086.1,...,-24.8112,-2057.62,-1228.54,769.634,-3378.34,-3348.2,-4163.03,-155.982,-3487.89,-1699.64
intensity6,-662.072,1775.15,-393.005,-1200.19,2086.17,1655.7,-3323.75,4788.87,-7699.29,-21299.5,...,-881.483,442.71,-864.756,-3467.58,1904.38,-1352.32,1814.82,-2634.11,2018.22,129.139
intensity7,850.946,-3081.54,-1987.27,-1229.99,-1195.83,-265.69,-3852.7,45767.1,-7954.25,-19783.7,...,16.9312,-4277.93,-2511.03,790.194,-3683.63,984.468,-24.5035,-28.1553,9494.26,-987.268
intensity8,1169.21,-727.655,4003.8,-485.068,1166.09,617.978,6093.15,43845.5,-6519.51,-8349.06,...,-419.586,-3532.48,-5007.73,-1218.56,-2879.9,2408.59,1836.54,2882.87,-5149.19,2130.76
